# Introduction to relational databases and integration with python

## What are relational databases 

As the term suggests relational databases are first and foremost databases.

Without technical knowledge, one could define a database as an abstract space to store and fetch data

Formally, a database is defined as a collection of structured information that has a specific querying language for both READ and WRITE instructions.


Since the information inside a database is structured, that means it follows a certain schema, plan, or method of design and implementation.

Best practices in the lates 1950s to the 1970s led to the creation of a formal mathematical way to design a database.

That model called <b> the relational model </b> is the product of the mathematical concept known as relational algebra, wich uses algebraic structures to define relationships and also provides a query language to read and update the data.

Relational algebra was founded by [Edgard F. Codd](https://en.wikipedia.org/wiki/Edgar_F._Codd)

In this course, we will not be tackling the details of relational algebra, as there will be a course you will attend (if you're a CCE major) that will provide deep understanding of this concept.

If you want to learn more about the mathematics behind the relational model you can read the following books:

<ul>
    <li> Database Design and Relational Theory - C.J. Date
    <li> Usage-Driven Database Desgin - George Tillman
    <li> Theory of relational Databases - David Maier
</ul>
  

## SQLite - definition and installation.

A RDMS (relational database management system) usually consists of 3 major components:

<ul>
    <li> Database server
    <li> Database client (mysql workbench, datagrip, CLI, many more...)
    <li> Query language (SQL - SQLite customized)
</ul>

### Step 1 - What is SQLite

For the sake of simplicity, as this chapter is just a basic introduction into the world of databases, we will not be installing a database server.

Our database will be stored in a database file (just a simple binary file).
As clients of the database system, we do not care for now, how the database is implemented. What a developer generally cares about, is how to connect to the database, and perform queries (READ and WRITE) operations, which will we cover here.

SQLite will provide us with an out of the box interface which we will use to create a database, and perform queries to it.

### Step 2 - Setting up SQLite

Modern verions of python (2.6x) and later, come with a builtin module to support SQLite.
No need to install anything.

To test if you have the module, you can always launch your python interpreter and import the sqlite3 module:

```python
import sqlite3
```

If you have issues importing, try installing it via pip. (pip install pysqlite3)

## Creating your first Schema

Simply speaking, a database schema defines the different relations and the relationships between them. A schema is usually defined in a formal language - SQL in this case. 


A schema can also be referred to as database in some terminologies, however a schema is a more accurate than as database can be confused with the database server itself.

### What are relations ?

Relations are a set of what is called named attribute values. An attribute is a name paired with a domain (nowadays more commonly referred to as a type or data type).

In modern database systems, a table is the physical representation of the relation. Named attribute values are the columns of that table, they have a name and a type associated with them.

### Concrete example:

We're going to create a simple yet rich example to learn more about relations

We're going to consider the following scenario: we are creating a social network application and we are in charge of creating the part related to posts and post comments

#### Reasoning about the structure

The most intuitive thing would be to create a relation that contains some basic information, for example we have the following *post* relation

| Field Name  | Field Type  |
| ----------- | ----------- |
| Title       | varchar(100)|
| Caption     | varchar(500)|
| Image       | binary(1K)  |
| CreatedAt   | Timestamp   |
| SubmittedBy | varchar(500)|
| Likes       | integer     |

Looking at this structure, we can clearly see that we have a certain problem of identifying the post:

What if we had the same post title, same caption and the same image submitted by two people with the same username.

Each tuple in the relation, must be uniquely identified. If there is nothing natevily unique about the studied entity then we stumble upon the ID field.

You, a student in USJ are identified by an ID, because no matter which angle you look at, there is nothing in your basic info that would make you uniquely identifiable...

Thus, we have the updated structure:

| Field Name  | Field Type  |
| ----------- | ----------- |
| ID          | integer (AT)|
| Title       | varchar(100)|
| Caption     | varchar(500)|
| CreatedAt   | Timestamp   |
| SubmittedBy | varchar(100)|
| Likes       | integer     |


For the comments we'll keep it simple, a post comment only contains text and is associated to a post:


| Field Name  | Field Type  |
| ----------- | ----------- |
| ID          | integer (AT)|
| Content     | varchar(200)|
| CreatedAt   | Timestamp   |
| SubmittedBy | varchar(100)|
| PostID      | integer     |

Notice that here we have added the PostID Field to indicate that this comment is mapped to a certain post


To create these relation in actual tables we need to do that in SQL:


```SQL
   Create table post (
       ID integer auto_increment primary key,
       Title varchar(100),
       Caption varchar(500),
       CreatedAt timestamp,
       SubmittedBy varchar(100),
       Likes integer);
       
   Create table post_comment(
       ID integer auto_increment primary key,
       Content varchar(500),
       CreatedAt timestamp,
       SubmittedBy varchar(100),
       PostID integer,
       
       Constraint FOREIGN KEY (PostID) REFERENCES post(ID) on DELETE CASCADE
       );
```

In order to insert some records into the post table we can execute the SQL INSERT command:

```SQL
    INSERT INTO post(Title,Caption,CreatedAt,SubmittedBy,Likes)
    VALUES
        ('Title1','Caption1','2022-01-01 00:00:00','User1',0),
        ('Title2','Caption2','2022-01-01 00:00:00','User1',0),
        ('Title3','Caption3','2022-01-01 00:00:00','User1',0),
        ('Title4','Caption4','2022-01-01 00:00:00','User2',0),
        ('Title5','Caption5','2022-01-01 00:00:00','User2',0),
        ('Title6','Caption6','2022-01-01 00:00:00','User3',0)
```

Notice how we did not manually add the ID of the post. We don't have to since we specified that the post table assigns automatically an Integer ID that starts at 1 and increments with each record inserted.

We also insert some content into the comments table:

```SQL
    INSERT INTO post_comment(Content, CreatedAt, SubmittedBy,PostID)
    VALUES
        ('Content of comment related to post1', '2022-01-02 00:00:00', 'User1', 1),
        ('Content of another comment related to post1', '2022-01-01 00:00:00', 'User2', 1),
        ('Content of comment related to post2', '2022-01-01 00:00:00','User1', 2)
```




## Step 3 - SQLite module

As previously mentioned, the sqlite3 module provides us with ways to create a database and interact with it.

SQLite gives us the option to either persist our information in a binary or in the memory (RAM).
Since we want our information to remain persisent after memory release (shutdown), we opted to go with the file option (disk persistence).

```python
import sqlite3

#The connect method of sqlite3 will create a file called db_test.db (if not existent).
#It will use that file to store the data we insert to it.
#The returned connection object is an object that wraps the connection attributes (host,name,properties,etc...)
connection = sqlite3.connect("db_test.db")

#We then create a cursor object that will allow us to execute statements to the database.

cursor = connection.cursor()

#From there we can simply execute any SQL statement that we want.
#We can start by creating the tables mentioned above:


#Creating the post table
cursor.execute ( 
    '''
       Create table post (
       ID integer primary key autoincrement,
       Title varchar(100),
       Caption varchar(500),
       CreatedAt timestamp,
       SubmittedBy varchar(100),
       Likes integer);
    '''
)

#Filling the post table with the some dummy data
cursor.execute('''
    INSERT INTO post(Title,Caption,CreatedAt,SubmittedBy,Likes)
    VALUES
        ('Title1','Caption1','2022-01-01 00:00:00','User1',0),
        ('Title2','Caption2','2022-01-01 00:00:00','User1',0),
        ('Title3','Caption3','2022-01-01 00:00:00','User1',0),
        ('Title4','Caption4','2022-01-01 00:00:00','User2',0),
        ('Title5','Caption5','2022-01-01 00:00:00','User2',0),
        ('Title6','Caption6','2022-01-01 00:00:00','User3',0)
    ''')


#Creating the post_comment table
cursor.execute( '''
   Create table post_comment(
       ID integer primary key autoincrement,
       Content varchar(500),
       CreatedAt timestamp,
       SubmittedBy varchar(100),
       PostID integer,
       
       FOREIGN KEY (PostID) REFERENCES post(ID) on DELETE CASCADE
       );
    ''')

#Filling the post_comment table with dummy data
cursor.execute(''' INSERT INTO post_comment(Content, CreatedAt, SubmittedBy,PostID)
    VALUES
        ('Content of comment related to post1', '2022-01-02 00:00:00', 'User1', 1),
        ('Content of another comment related to post1', '2022-01-01 00:00:00', 'User2', 1),
        ('Content of comment related to post2', '2022-01-01 00:00:00','User1', 2)
    ''')



```

You can also read from the database with the same cursor. 
When performing read operations, the cursor will return an iterable which consitutes your solution.

```python
#Reading the post table contents:

#The returned object is a cursor object, that is also an iterable.
#Since it is an iterable, you can at any point convert it into a list
data = cursor.execute("SELECT * from post")

#Converting data into a list:

data = list(data)

```

## Step 4: Reading from the database: SELECT queries


The previously executed query:

```SQL
SELECT * FROM post;
```
is not very sophisticated, as it will return all the content of the table post.

There are extremely numerous scenarios where we do not want to read the whole post table:

<ul>
    <li>There are fields that we dot not want to include (Post ID for example)</li>
    <li>The post data contains too many rows, making the query extremely slow</li>
    <li>We want to return rows that meet certrain criteria</li>
    <li>We want to return an aggregation of a certain column in post. (Count the number of rows that meet a certain criteria) </li>
    <li>We want to combine post data with post_comment data in 1 return</li>
</ul>

We might to need to solve one or a combination of these issues at the same time in the same query.

Luckily SQL provides us with keywords to optimize/customize our query:

*There are fields that we dot not want to include (Post ID for example)*: can be solved by only selecting the wanted fields: ( SELECT Title Caption from post )

*The post data contains too many rows, making the query extremely slow*: we can paginate the returned data, by using an offset and a limit.

*We want to return rows that meet certrain criteria*: using the WHERE clause, we can filter the results based on one or multiple criteria: (SELECT * from post where Title = 'Introduction to C++')
*We want to return an aggregation of a certain column in post. (Count the number of rows that meet a certain criteria)*: Use aggregation functions ( SELECT COUNT(*) from post; )

*We want to combine post data with post_comment data in 1 return* : Use join statements : (SELECT * from post left join post_comment on post.ID = post_comment.PostID)

## Drawbacks and limits of the previously shared method:

Using the mysql connector and the cursor to execute queries and to read from the database is extremely easy to achieve, however may not be the best thing to do:

Writing your own native queries and without wrapper objects can lead to:

<ul>
    <li>Complicated SQL queries</li>
    <li>Prone to security vulnerabilities such as SQL injection</li>
    <li>No Wrapper objects, not type sound</li>
    <li>Harder to parallel call the database </li>
    <li>Slower development time, as we need to write TupleToClass Functions all the time </li>
</ul>

## ORMs (Object relational Mappers)

Object–relational mapping in computer science is a programming technique for converting data between incompatible type systems using object-oriented programming languages. This creates, in effect, a "virtual object database" that can be used from within the programming language

ORMs remove the need for us to convert our query results from tuples to objects, and remove the need for us write our queries by hands, thus reducing the development time and also allow us to leave the implementation for specialized librairies.

Take a look at SqlAlchemy or Django